In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import time

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

### Preprocessing of the images

In [ ]:
%matplotlib inline

def normalize_image(img):
    means = np.mean(img, axis=(0, 1))
    means = means[None,:]
    
    std = np.std(img, axis=(0, 1))
    std = std[None,:]
    return (img - means) / std

def preprocess_image(img):
    img_crop = img[56:150, :, :]
    img_normed = normalize_image(img_crop)
    return img_normed

### Helper Class to load data and generate batches

In [ ]:
class DataHelper:
    def __init__(self, data_file):
        xs = []
        ys = []
        
        with open(data_file) as f:
            header = f.readline()
            dirname = os.path.dirname(data_file)
            for line in f:
                fields = line.split(", ")
                # Loading only the center images
                xs.append(os.path.join(dirname, fields[0]))
                ys.append(fields[3])
                
        c = list(zip(xs, ys))
        random.shuffle(c)
        xs, ys = zip(*c)
        
        self._batch_pointer = 0
        self._train_xs = xs[:(int)(len(xs) * 0.9)]
        self._train_ys = ys[:(int)(len(xs) * 0.9)]

        val_xs = []
        val_ys = []
        val_size = (int)(len(xs) * 0.1)
        for i in range(val_size):
            img = plt.imread(xs[-i])
            img_pre = preprocess_image(img)
            
            val_xs.append(img_pre)
            val_ys.append(ys[-i])
        
        self._val_xs = np.asarray(val_xs)
        self._val_ys = np.asarray(val_ys)
        
    def data_size(self):
        return len(self._train_ys) + len(self._val_ys)
    
    def val_data(self):
        return self._val_xs, self._val_ys

    def next_train_batch(self, batch_size):
        x_out = []
        y_out = []
        for i in range(batch_size):
            data_idx = (self._batch_pointer + i) % len(self._train_ys)
            
            img = plt.imread(self._train_xs[data_idx])
            img_pre = preprocess_image(img)
            
            x_out.append(img_pre)            
            y_out.append(self._train_ys[data_idx])

        self._batch_pointer += batch_size
        return np.asarray(x_out), np.asarray(y_out)

In [ ]:
dh = DataHelper('data/driving_log.csv')
print(dh.data_size())

In [ ]:
x, y = dh.next_train_batch(64)
print(x.shape, y.shape)

In [ ]:
idx = random.randint(0, len(y) - 1)
img = x[idx]
print(img.shape)
plt.imshow(img, cmap='gray')

In [ ]:
def simple_conv():
    model = Sequential()
    
    model.add(Convolution2D(24, 5, 5, subsample=(2, 2), input_shape=(94, 320, 3),
                            activation='relu', init='he_normal'))
    model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu', init='he_normal'))
    model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu', init='he_normal'))
    
    model.add(Convolution2D(64, 5, 5, activation='relu', init='he_normal'))
    model.add(Convolution2D(64, 5, 5, activation='relu', init='he_normal'))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(1164, activation='relu', init='he_normal'))    
    model.add(Dropout(0.5))    
    model.add(Dense(100, activation='relu', init='he_normal'))
    model.add(Dense(50, activation='relu', init='he_normal'))
    model.add(Dense(10, activation='relu', init='he_normal'))
    model.add(Dense(1, activation='tanh', init='he_normal'))

    opt = Adam(lr=0.0001)
    model.compile(optimizer=opt, loss='mean_squared_error')

    return model

In [ ]:
model = simple_conv()
model.summary()

In [ ]:
last_layer = model.layers[-1].get_weights()
second_last_layer = model.layers[-2].get_weights()

print("Second last layer")
print(second_last_layer[0][:2])
print("output")
print(second_last_layer[1])

print("Last layer")
print(last_layer[0].reshape(10))
print("final output")
print(last_layer[1])

In [13]:
EPOCHS = 1
BATCH_SIZE = 64
DATA_SIZE = dh.data_size()

val_x, val_y = dh.val_data()

print("Starting training")
for steps in range(60):
    step_start = time.time()
    cur_epoch = steps / DATA_SIZE
    x, y = dh.next_train_batch(BATCH_SIZE)

    train_loss = model.train_on_batch(x, y)
    val_loss = model.evaluate(val_x, val_y, verbose=0)
    
    time_taken = time.time() - step_start
    print("Steps {} train loss {:0.6f} validation loss {:0.6f} time taken {:0.1f}s".format(
            steps, train_loss, val_loss, time_taken))
    
    model.save("model.h5")

Starting training
Steps 0 train loss 0.031671 validation loss 0.016157 time taken 1.2s
Steps 1 train loss 0.022991 validation loss 0.016154 time taken 1.3s
Steps 2 train loss 0.021220 validation loss 0.016148 time taken 1.2s
Steps 3 train loss 0.012573 validation loss 0.016141 time taken 1.1s
Steps 4 train loss 0.038619 validation loss 0.016135 time taken 1.1s
Steps 5 train loss 0.024697 validation loss 0.016133 time taken 1.1s
Steps 6 train loss 0.025924 validation loss 0.016130 time taken 1.1s
Steps 7 train loss 0.016732 validation loss 0.016123 time taken 1.1s
Steps 8 train loss 0.017975 validation loss 0.016114 time taken 1.2s
Steps 9 train loss 0.020340 validation loss 0.016104 time taken 1.2s
Steps 10 train loss 0.017827 validation loss 0.016094 time taken 1.2s
Steps 11 train loss 0.015686 validation loss 0.016083 time taken 1.1s
Steps 12 train loss 0.034869 validation loss 0.016075 time taken 1.2s
Steps 13 train loss 0.037945 validation loss 0.016069 time taken 1.1s
Steps 14 tra

In [14]:
test_idx = np.random.randint(0, len(val_y), size=10)
y_pred = model.predict(val_x[test_idx, ])
print(y_pred.reshape(10))
print(val_y[test_idx,])

[ 0.01934147 -0.00740436  0.00159991  0.0039825   0.01777484  0.0039825
  0.00078883 -0.00402864  0.00298586  0.0039825 ]
['0.100034' '-0.09773462' '0' '-0.2021725' '0.0904655' '0' '0' '-0.0787459'
 '0' '0']


In [15]:
last_layer = model.layers[-1].get_weights()
second_last_layer = model.layers[-2].get_weights()

print("Second last layer")
print(second_last_layer[0][:2])
print("output")
print(second_last_layer[1])

print("Last layer")
print(last_layer[0].reshape(10))
print("final output")
print(last_layer[1])

Second last layer
[[ 0.26945588 -0.05191147 -0.3705292  -0.26706398  0.10041998  0.1699838
   0.1535826  -0.16913702  0.08579066  0.2918222 ]
 [-0.13610914 -0.16321753 -0.13842998  0.07048266  0.25611222 -0.04282611
   0.02490786  0.19878256  0.17138635  0.05657545]]
output
[-0.00069582 -0.00348687 -0.00511459 -0.00334825 -0.00389479 -0.00470362
 -0.00225571 -0.00228998 -0.00367564 -0.00364226]
Last layer
[ 0.11915189  0.60039246 -0.2546992   0.53063667 -0.46422622 -0.06258549
  0.25730845 -0.20006031 -1.58154202  0.3335906 ]
final output
[ 0.00398253]
